Setup

In [53]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import TensorBoard

# Create a TensorBoard callback
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1)

Read and transform the data

In [45]:
# load dataset
CCData = pd.read_csv("Credit_card.csv")
CCData2 = pd.read_csv("Credit_card_label.csv")
X = CCData[["Ind_ID","GENDER","Car_Owner","Propert_Owner","CHILDREN","Annual_income","Type_Income","EDUCATION",
        "Marital_status","Housing_type","Birthday_count","Employed_days","Mobile_phone","Work_Phone","Phone",
        "EMAIL_ID","Type_Occupation","Family_Members"]]
Y = CCData2[["label"]]

#Find and replace all 'NaN data
#Columns with string data get "N/A"
#Columns with numbered data get 0
listNan = X.loc[:, X.isna().any()]
print('Columns with NaN data:')
print(list(listNan))
X['GENDER'] = X['GENDER'].fillna('N/A')
X['Type_Occupation'] = X['Type_Occupation'].fillna('N/A')
X['Annual_income'] = X['Annual_income'].fillna(0)
X['Birthday_count'] = X['Birthday_count'].fillna(0)

#Encode the catagorical data
le = LabelEncoder()
X['GENDER'] = le.fit_transform(X.GENDER.values)
X['Car_Owner'] = le.fit_transform(X.Car_Owner.values)
X['Propert_Owner'] = le.fit_transform(X.Propert_Owner.values)
X['Type_Income'] = le.fit_transform(X.Type_Income.values)
X['EDUCATION'] = le.fit_transform(X.EDUCATION.values)
X['Marital_status'] = le.fit_transform(X.Marital_status.values)
X['Housing_type'] = le.fit_transform(X.Housing_type.values)
X['Type_Occupation'] = le.fit_transform(X.Type_Occupation.values)

# Standardizing the features and perform pca
X = StandardScaler().fit_transform(X)
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)

# Split the dataset into training and test sets
X, X_test, Y, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler().fit(X)
X = scaler.transform(X)
X_test = scaler.transform(X_test)

#Turn Y into a matrix
Y = np.ravel(Y)
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)

Columns with NaN data:
['GENDER', 'Annual_income', 'Birthday_count', 'Type_Occupation']


Create and train the model

In [69]:
#Create the model
model = Sequential()
model.add(Dense(4, input_dim=X.shape[1], activation='sigmoid', kernel_initializer=HeNormal()))
model.add(Dense(3, activation='sigmoid'))
model.add(Dense(2, activation='sigmoid'))  # Output layer
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=0.01),
              metrics=['accuracy'])
#Training the model
history = model.fit(X, Y, epochs=200, batch_size=32, validation_data=(X_test, y_test), callbacks=[tensorboard])

Epoch 1/200
39/39 [==============================] - 1s 8ms/step - loss: 0.3758 - accuracy: 0.8829 - val_loss: 0.3172 - val_accuracy: 0.9032
Epoch 2/200
39/39 [==============================] - 0s 4ms/step - loss: 0.3590 - accuracy: 0.8829 - val_loss: 0.3204 - val_accuracy: 0.9032
Epoch 3/200
39/39 [==============================] - 0s 4ms/step - loss: 0.3572 - accuracy: 0.8829 - val_loss: 0.3171 - val_accuracy: 0.9032
Epoch 4/200
39/39 [==============================] - 0s 4ms/step - loss: 0.3578 - accuracy: 0.8829 - val_loss: 0.3148 - val_accuracy: 0.9032
Epoch 5/200
39/39 [==============================] - 0s 4ms/step - loss: 0.3554 - accuracy: 0.8829 - val_loss: 0.3163 - val_accuracy: 0.9032
Epoch 6/200
39/39 [==============================] - 0s 5ms/step - loss: 0.3555 - accuracy: 0.8829 - val_loss: 0.3138 - val_accuracy: 0.9032
Epoch 7/200
39/39 [==============================] - 0s 4ms/step - loss: 0.3535 - accuracy: 0.8829 - val_loss: 0.3159 - val_accuracy: 0.9032
Epoch 8/200
3

KeyboardInterrupt: 

**1: Adam Optimizer with learning rate = 0.01**
1.   loss='sparse_categorical_crossentropy', Accuracy = 0.9136
2.   loss='mse', Accuracy = 0.2165
3.   loss='msle', Accuracy = 0.5468
4.   loss = 'poisson', Accuracy = 0.3586
5.   loss='binary_crossentropy', Accuracy = 0.9071




